In [13]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import pickle

In [14]:
path = 'IP_Project_DataBase'

In [3]:
os.listdir(path)

['2110992492.jpg',
 '2110992491.jpg',
 '2110992490.jpg',
 '2110992461.jpg',
 '2110992459.jpg']

In [15]:
images = []
classNames = []
mylist = os.listdir(path)
for cl in mylist:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [16]:
classNames

['2110992492', '2110992491', '2110992490', '2110992461', '2110992459']

In [17]:
def findEncodings(images):
    img_count = 0
    encodeList = []
    for idx, img in enumerate(images):
        if img is None:
            print(f"Image at index {idx} is not loaded correctly.")
            continue
        
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            encoded_faces = face_recognition.face_encodings(img)
            if encoded_faces:
                encodeList.append(encoded_faces[0])
            else:
                print(f"No face encodings found in image at index {idx}.")
        except Exception as e:
            print(f"An error occurred with image at index {idx}: {e}")
            continue
        
        img_count += 1
        print(f"Processed {img_count} images.")
        
    return encodeList
encoded_face_train = findEncodings(images)

Processed 1 images.
Processed 2 images.
Processed 3 images.
Processed 4 images.
Processed 5 images.


In [18]:
with open('processed_images.pickle', 'wb') as f:
    pickle.dump(encoded_face_train, f)

In [19]:
with open('processed_images.pickle', 'rb') as f:
    encoded_face_train = pickle.load(f)

In [20]:
def markAttendance(name):
    print("doing attendance")
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            time = now.strftime('%I:%M:%S:%p')
            date = now.strftime('%d-%B-%Y')
            f.writelines(f'{name}, {time}, {date} \n')

In [21]:
attendance_arr = []
count_frame = 0
count_flag = 0
cap = cv2.VideoCapture(0)

try:
    while True:
        success, img = cap.read()
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
        faces_in_frame = face_recognition.face_locations(imgS)
        encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

        for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
            matches = face_recognition.compare_faces(encoded_face_train, encode_face)
            faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
            matchIndex = np.argmin(faceDist)
            name = "unknown"
            if matches[matchIndex]:
                name = classNames[matchIndex]
                if min(faceDist) > 0.5:
                    name = "unknown"
            
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)

            if count_flag <= 0:
                count_flag = 1
                flag = False

            attendance_arr.append(name)
            if len(attendance_arr) == 10:
                maxi = 0
                res = attendance_arr[0]
                for i in attendance_arr:
                    freq = attendance_arr.count(i)
                    if freq > maxi:
                        maxi = freq
                        res = i
                attendance_arr.clear()
                if res != "unknown":
                    # Implement your markAttendance function accordingly
                    markAttendance(res)
                    flag = True
                    count_flag = 10

            if flag:
                if name != "unknown":
                    cv2.putText(img, f"Attendance Done of {name}", (x1 - 200, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            else:
                if name == "unknown":
                    cv2.putText(img, "Not Authorized", (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                else:
                    cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

        count_frame += 1
        count_flag -= 1
        cv2.imshow('webcam', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
except:
    print("Something went wrong")
cap.release()
cv2.destroyAllWindows()

doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
doing attendance
Something went wrong


In [22]:
import pandas as pd

In [23]:
df = pd.read_csv("Attendance.csv")

In [24]:
df.tail()

,Roll Number,Time,Date
0,2110992492,05:22:15:PM,23-May-2024
1,2110992491,05:22:16:PM,23-May-2024
2,2110992461,05:22:16:PM,23-May-2024
